### Portfolio Factor Analysis

In [3]:
import numpy as np
import pandas as pd

Read in Excel file and specifically import net returns data

In [11]:
xls=pd.ExcelFile('Simple_Strategies_Returns.xlsx')
netreturns = pd.read_excel(xls, 1)

Split data into 'new' and 'old' sections:  
'Old' section has 120 months of 26 factors  
'New' section has 486 months of 34 factors

In [35]:
oldnetreturns = netreturns[netreturns.Month < 197307]
oldnetreturns = oldnetreturns.reset_index()
oldnetreturns = oldnetreturns.dropna(axis=1)

In [36]:
oldnetreturns.shape

(120, 26)

In [20]:
newnetreturns = netreturns[netreturns.Month >= 197307]
newnetreturns = newnetreturns.reset_index()

In [32]:
newnetreturns.shape

(486, 34)

TODO:  
- Import S&P 500 returns  
- Import Fama-French data
- Build dimensionality reduction pipeline (to try different methods)
- Build supervised learning pipeline (i.e., timeseries regression of reduced data on S&P returns)